In [1]:
%matplotlib widget

In [2]:
from fimpylab import LightsheetExperiment

In [11]:
from pathlib import Path
import numpy as np
from matplotlib import  pyplot as plt
import flammkuchen as fl
import pandas as pd

from fimpylab import autoload_experiment

import pandas as pd

import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Libertinus Sans']

# %autoreload
from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, \
    fictive_trajectory_and_fit, qap_sorting_and_phase, fit_phase_neurons


In [4]:
plt.close("all")
# path = Path("/Users/luigipetrucco/Desktop/source_data_batch1/210729_f6_cwccwck")
path = Path("/Users/luigipetrucco/Desktop/hagar_data/210810_f0")
traces = fl.load(path / "filtered_traces.h5", "/detr")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LightsheetExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

In [24]:
selected = fl.load(path / "selected.h5")

stim_start = traces.shape[0] // 2
t_lims = (100 * fn, stim_start)
t_slice = slice(*t_lims)

pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [25]:
# plt.close("all")
mot_t_slice = slice(stim_start, traces.shape[0])
f, axs = plt.subplots(1, 3, figsize=(7., 3.), sharex=True, sharey=True)
# plt.subplot(1,2,1)
# phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
axs[0].plot(pcaed[t_slice, 0], pcaed[t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[0].scatter(pcaed[t_slice, 0], pcaed[t_slice, 1], 
                 c=phase[t_slice], lw=0.5, s=5, cmap="twilight",) 

axs[1].plot(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[1].scatter(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
                 c=phase[mot_t_slice], lw=0.5, s=5, cmap="twilight",) 

axs[2].plot(pcaed[:, 0], pcaed[:, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[2].scatter(pcaed[:, 0], pcaed[:, 1], 
                 c=phase[:], lw=0.5, s=5, cmap="twilight",) 
sns.despine()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
phases_neuron, _ = fit_phase_neurons(traces[t_slice, selected], phase[t_slice])
perm = np.argsort(phases_neuron)

100%|██████████| 391/391 [00:00<00:00, 519.81it/s]


In [23]:
from bouter.utilities import reliability

In [14]:
def reshape_for_reliability(traces, n_reps):
    n_rep_pts = traces.shape[0] // n_reps
    return np.swapaxes(traces.reshape(n_reps, n_rep_pts, -1), 0, 1)
    

def reliability_from_traces(traces, n_reps):
    """Crop traces and returns reliability scores, assuming
    that the traces consist of consecutive blocks of a stimulus
    presented n_reps times.
    """
    resh = reshape_for_reliability(traces, n_reps)
    return reliability(resh)

In [36]:
rels = reliability_from_traces(traces[stim_start:, :], 4)
reshaped = reshape_for_reliability(traces[stim_start:, :], 4)

stim_sel = np.argwhere(rels > 0.6)[:, 0]

In [28]:
plt.close("all")
plt.figure(figsize=(4, 2))
plt.plot(reshaped[:, :, np.argsort(rels)[-1]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
plt.figure(figsize=(4, 3))
plt.hist(rels, np.arange(0, 1, 0.04), lw=0, alpha=0.4, density=True, label="all ROIs")
plt.hist(rels[selected], np.arange(0, 1, 0.04), lw=0, alpha=0.4, density=True, label="ring")
plt.ylabel("Count")
plt.xlabel("Reliability")
plt.show()
sns.despine()
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")

In [37]:
plt.figure(figsize=(4, 4))
plt.scatter(coords[:, 1], coords[:, 2], c=(0.9,)*3)
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
#            c=np.linspace(-np.pi, np.pi, perm_stim.max()+1)[np.argsort(perm_stim)] , cmap="twilight")
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
#            c=rels[stim_sel], cmap="viridis")
plt.scatter(coords[selected, 1], coords[selected, 2],  
            facecolors='none', edgecolors=cols[3], lw=0.5)
plt.scatter(coords[selected, 1], coords[selected, 2],  
            c=np.linspace(-np.pi, np.pi, perm.max()+1)[np.argsort(perm)] , cmap="twilight")
plt.axis("equal")
plt.axis("off")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.


(-11.227083333333333,
 245.54652777777778,
 -11.62717105263158,
 306.1179605263158)

In [41]:
plt.figure(figsize=(4, 4))
plt.scatter(coords[:, 1], coords[:, 0]*10, c=(0.9,)*3)
plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
            c=rels[stim_sel], cmap="viridis")
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 0]*10,  
#            c=rels[stim_sel], cmap="viridis")
plt.scatter(coords[selected, 1], coords[selected, 2],  
            facecolors='none', edgecolors=cols[3], lw=0.5)
#plt.scatter(coords[selected, 1], coords[selected, 2],  
#            c=np.linspace(-np.pi, np.pi, perm.max()+1)[np.argsort(perm)] , cmap="twilight")
plt.axis("equal")
plt.axis("off")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.


(-11.227083333333333,
 245.54652777777778,
 -14.415714285714285,
 302.72999999999996)

In [42]:
 perm_stim, com_phase = qap_sorting_and_phase(traces[:, stim_sel], t_lims=(5100, 10200))

plt.figure(figsize=(7, 3))
plt.subplot(121)
plt.imshow(np.corrcoef(traces[5100:, stim_sel].T)[perm_stim, :][:, perm_stim], 
           vmax=1, vmin=-1, cmap="RdBu_r")

plt.subplot(122)
plt.imshow(traces[:, stim_sel[perm_stim]].T, cmap="gray_r", aspect="auto", vmin=-2, vmax=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …